In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
import skimpy

# IDÉES ETAPES À SUIVRES :
1) Exploration et préparation des données:

- Analyser les données pour comprendre leur structure.
- Nettoyer les données (traiter les valeurs manquantes, les doublons, etc.).
- Effectuer une analyse exploratoire des données (EDA).

2) Prétraitement des données:

- Normaliser ou standardiser les données.
- Encoder les variables catégorielles.
- Diviser les données en ensembles d'entraînement et de test.

3) Sélection du modèle:

- Choisir les algorithmes de machine learning appropriés.
- Entraîner plusieurs modèles pour comparer leurs performances.

4) Entraînement du modèle:

- Entraîner les modèles sur l'ensemble d'entraînement.
- Ajuster les hyperparamètres pour optimiser les performances.

5) Évaluation du modèle:

- Évaluer les modèles sur l'ensemble de test.
- Utiliser des métriques de performance appropriées (précision, rappel, F1-score, etc.).


In [11]:
train_df = pd.read_csv('classification-multi-classes/train.csv', 
                      delimiter = ",",
                      header = 0)

train_df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1055.0,NaN,19724.0,F,NaN,NaN,NaN,N,1.3,NaN,3.64,NaN,NaN,NaN,NaN,209.0,10.5,3.0,C
1,1,3282.0,Placebo,17884.0,F,N,Y,Y,N,0.7,309.0,3.60,96.0,1142.0,71.3,106.0,240.0,12.4,4.0,C
2,2,1653.0,NaN,20600.0,F,NaN,NaN,NaN,N,2.2,NaN,3.64,NaN,NaN,NaN,NaN,139.0,9.5,2.0,C
3,3,999.0,D-penicillamine,22514.0,F,N,Y,N,N,1.0,498.0,3.35,89.0,1601.0,164.3,85.0,394.0,9.7,3.0,C
4,4,2202.0,NaN,17897.0,F,NaN,NaN,NaN,N,17.2,NaN,3.15,NaN,NaN,NaN,NaN,432.0,11.2,3.0,C


In [12]:
train_df.shape

(15000, 20)

## <span style="color:red">1.  Exploration et préparation des données</span>

- id : Identifiant unique de chaque patient ou enregistrement. 
- N_Days : Nombre de jours écoulés depuis un événement de référence (par exemple, diagnostic, début du suivi, ou début du traitement). Début du traitement 1986.
- Drug : Nom ou catégorie du médicament administré au patient.
- Age : Âge du patient (probablement en années).
- Sex : Sexe du patient (Male ou Female).
- Ascites : Présence d’ascite (accumulation de liquide dans la cavité abdominale). Valeurs possibles : Yes, No, ou NA (inconnue).
- Hepatomegaly : Présence d’hépatomégalie (augmentation du volume du foie). Valeurs possibles : Yes, No, ou NA.
- Spiders : Présence de telangiectasies en araignée (lésions vasculaires cutanées). Valeurs possibles : Yes, No, ou NA.
- Edema : Présence d’œdème. Peut inclure des catégories comme None, Slight, ou Marked.
- Bilirubin : Niveau de bilirubine dans le sang (en mg/dL), un marqueur de la fonction hépatique.
- Cholesterol : Niveau de cholestérol dans le sang (en mg/dL), souvent indicatif d’une santé globale ou de troubles métaboliques.
- Albumin : Niveau d’albumine dans le sang (en g/dL), un marqueur nutritionnel et de la fonction hépatique.
- Copper : Concentration de cuivre dans le sang ou l’urine (en µg/dL), indicateur d’une éventuelle maladie de Wilson ou autre désordre métabolique.
- Alk_Phos : Niveau de phosphatases alcalines (en UI/L), un indicateur de fonction hépatique ou osseuse.
- SGOT : Niveau d'enzymes transaminases (SGOT ou AST) (en UI/L), indicateur de dommages au foie ou à d’autres organes.
- Tryglicerides : Niveau de triglycérides dans le sang (en mg/dL), un marqueur lipidique et métabolique.
- Platelets : Nombre de plaquettes dans le sang (en milliers/μL), un indicateur de la coagulation.
- Prothrombin : Temps de prothrombine (en secondes ou % d’activité), un test de coagulation pour évaluer la fonction hépatique.
- Stage : Stade de la maladie, basé sur un système de classification (souvent de 1 à 4, où 4 est le plus sévère).
- Status : État ou issue du patient. Par exemple : Alive, Deceased, ou autres catégories. (VARIABLE CYBLE)

### <span style="color:blue">Types des variables</span>

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  int64  
 2   Drug           8564 non-null   object 
 3   Age            15000 non-null  float64
 4   Sex            15000 non-null  object 
 5   Ascites        8563 non-null   object 
 6   Hepatomegaly   8564 non-null   object 
 7   Spiders        8555 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6669 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8452 non-null   float64
 13  Alk_Phos       8558 non-null   float64
 14  SGOT           8555 non-null   float64
 15  Tryglicerides  6630 non-null   float64
 16  Platelets      14414 non-null  float64
 17  Prothrombin    14974 non-null  float64
 18  Stage 

In [14]:
# Mettre N_Days en int
train_df['N_Days'] = train_df['N_Days'].astype('int')

# stage mettre en int
train_df['Stage'] = train_df['Stage'].astype('object')

In [ ]:
train_df.describe()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,15000.000000,15000.000000,15000.000000,15000.000000,6669.000000,15000.000000,8452.000000,8558.000000,8555.000000,6630.000000,14414.000000,14974.000000,15000.000000
mean,7499.500000,1966.998733,19248.502373,1.861462,327.670115,3.525314,74.715689,1667.605282,106.579545,110.297097,253.822027,10.627748,3.025667
std,4330.271354,1355.897299,3697.546749,2.752375,179.801705,0.369981,74.211915,1861.746315,56.493338,49.676146,94.074989,0.741888,0.877796
min,0.000000,3.000000,133.300000,0.300000,120.000000,1.800000,4.000000,16.000000,8.000000,33.000000,0.400000,7.100000,1.000000
25%,3749.750000,1092.000000,16374.000000,0.600000,242.000000,3.290000,31.000000,721.000000,71.000000,80.000000,183.000000,10.100000,2.000000
50%,7499.500000,1785.000000,19544.000000,0.900000,280.000000,3.580000,52.000000,1072.000000,97.650000,99.000000,251.000000,10.600000,3.000000
75%,11249.250000,2635.000000,22369.000000,1.800000,360.000000,3.770000,86.000000,1691.250000,133.300000,133.000000,312.000000,11.000000,4.000000
max,14999.000000,40795.000000,35472.000000,28.000000,1790.000000,4.640000,588.000000,19270.000000,2148.000000,598.000000,724.000000,18.000000,4.000000


### <span style="color:blue">Valeurs manquantes</span>

In [ ]:
# Valeur manquante
train_df.isna().sum()/len(train_df)*100

id                0.000000
N_Days            0.000000
Drug             42.906667
Age               0.000000
Sex               0.000000
Ascites          42.913333
Hepatomegaly     42.906667
Spiders          42.966667
Edema             0.000000
Bilirubin         0.000000
Cholesterol      55.540000
Albumin           0.000000
Copper           43.653333
Alk_Phos         42.946667
SGOT             42.966667
Tryglicerides    55.800000
Platelets         3.906667
Prothrombin       0.173333
Stage             0.000000
Status            0.000000
dtype: float64

In [ ]:
# On constate que les pourcentages de valeurs manquantes sont assez proches pour certaines variables:
# - Ascites, Hepatomegaly, Spiders ont un pourcentage de valeurs manquantes quasiment égal
#   De plus, ces trois variables ont la meme structure (yes/no/na)
#   -> il est possible que ce soit les mêmes individus pour lesquels on a des NA (personnes qui n'ont juste 
#   pas repondus ou pas suivi le protocole)
# - Cholesterol, Tryglicerides ont des pourcentages de valeurs manquantes plus élevés que les autre variables
#   -> il s'agirait des meme individus qui n'ont pas suivi le traitement + des individus qui ne connaitraient
#    pas ces taux 

# => en generale les pourcenatge de valeur manquantes sur des variables qui semblent liées entre elles
# => il est possible que ce soit les mêmes individus pour lesquels on a des NA, donc un groupe qui n'aurait 
# pas suivi le protocole 
# IL FAUDRA FAIRE UNE ANALYSE PLUS POUSSEE POUR VOIR SI LES NA SONT PReSENTS SUR LES MEMES INDIVIDUS